In [ ]:
from __future__ import division
import numpy as np
import pandas as pd
import keras.preprocessing.image as ks_pre_img
import skimage.io as sk_io
import matplotlib.pyplot as plt

import skimage.transform as sk_tr



In [ ]:
train_df = pd.read_csv("../input/train.csv", index_col="id", usecols=[0])[0:10]

In [ ]:
# keras: 0..255 values (grauyscale)
train_df["ks_images"] = [np.array(ks_pre_img.load_img("../input/train/images/{}.png".format(idx),\
                                                      #interpolation = "nearest",\
                                                  grayscale=True)) for idx in train_df.index]
# skimage: 0 .. 1 values (3 colors flatten to one)
train_df["sci_image"] = [np.array(sk_io.imread("../input/train/images/{}.png".format(idx),\
                                               flatten=True,plugin='pil')) \
                      for idx in train_df.index]

In [ ]:
"keras:",train_df.ks_images[0].shape," skimage:",train_df.sci_image[0].shape

In [ ]:
im1 = (train_df["ks_images"][0]/255)

In [ ]:
im2 = (train_df["sci_image"][0])

In [ ]:
imgplot = plt.imshow(im1,cmap="gray")
plt.colorbar()

In [ ]:
imgplot = plt.imshow(im1)
plt.colorbar()

## warp an image